In [1]:
# Apache Spark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Python
import numpy as np
import pandas as pd
from itertools import product
import time

In [2]:
# Start a SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Bitcoin Prediction") \
    .getOrCreate()

sc = spark.sparkContext

In [12]:
# Read csv file
filename = "bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv"
dataset = spark.read.format("csv") \
          .option("inferSchema",'True') \
          .option("header",True) \
          .load(filename)

In [4]:
dataset.show(5)

+----------+----+----+----+-----+------------+------------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)| Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+------------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|2.0000000193000003|          4.39|
|1325317980|null|null|null| null|        null|              null|          null|
|1325318040|null|null|null| null|        null|              null|          null|
|1325318100|null|null|null| null|        null|              null|          null|
|1325318160|null|null|null| null|        null|              null|          null|
+----------+----+----+----+-----+------------+------------------+--------------+
only showing top 5 rows



### fill NaN values

In [5]:
# select cols to fill NaN
na_cols = dataset.columns
na_cols = na_cols[1:]
na_cols

['Open',
 'High',
 'Low',
 'Close',
 'Volume_(BTC)',
 'Volume_(Currency)',
 'Weighted_Price']

In [6]:
from pyspark.sql import Window

#### interpolation

In [7]:
def interpolation_fill(dataset,order_col, value_col):
    # fill not nan value with null to fetch the last/fisrt not null val
    #dataset = dataset.withColumn(value_col,F.when(F.isnan(F.col(value_col)),None).otherwise(F.col(value_col)))
    # idx_not_na -> not null value column's idx number
    dataset=dataset.withColumn('idx_not_na',F.when(F.col(value_col).isNotNull(),F.col("idx")))
    
    w_start = Window.orderBy(order_col).rowsBetween(Window.unboundedPreceding,-1)
    # start_val -> last not null value
    dataset = dataset.withColumn('start_val',F.last(value_col,True).over(w_start))
    # start_idx -> last not null value idx
    dataset = dataset.withColumn('start_idx',F.last('idx_not_na',True).over(w_start))
    
    w_end = Window.orderBy(order_col).rowsBetween(0,Window.unboundedFollowing)
    # end_val -> next not null value
    dataset = dataset.withColumn('end_val',F.first(value_col,True).over(w_end))
    # end_idx -> next not null value idx
    dataset = dataset.withColumn('end_idx',F.first('idx_not_na',True).over(w_end))
    
    # diff_idx -> numbers of missing value columns
    dataset = dataset.withColumn('diff_idx',F.col('end_idx')-F.col('start_idx'))
    dataset = dataset.withColumn('curr_idx',F.col('diff_idx')-(F.col('end_idx')-F.col('idx')))
    
    #fill the null values with the function
    lin_interp_func = (F.col('start_val')+(F.col('end_val')-F.col('start_val'))/F.col('diff_idx')*F.col('curr_idx'))
    dataset = dataset.withColumn(value_col,F.when(F.col(value_col).isNull(),lin_interp_func).otherwise(F.col(value_col)))
    
    # dropping all the unnecessary columns
    dataset = dataset.select([order_col]+na_cols+['idx'])
    return dataset

#add an idx column 9th
w = Window.orderBy('Timestamp')
dataset = dataset.withColumn('idx',F.row_number().over(w))

for value_col in na_cols:
    dataset = interpolation_fill(dataset, 'Timestamp', value_col)
dataset = dataset.drop('idx')

#### forward fill (out final method)

In [9]:
def fwfill(dataset,order_col, value_col):
    w_start = Window.orderBy(order_col).rowsBetween(Window.unboundedPreceding,-1)
    dataset = dataset.withColumn(value_col,F.when(F.col(value_col).isNull(),F.last(value_col,True).over(w_start)).otherwise(F.col(value_col)))
    dataset = dataset.select([order_col]+na_cols)
    return dataset
     
for value_col in na_cols:
    dataset = fwfill(dataset, 'Timestamp', value_col)

#### backward fill

In [11]:
def bwfill(dataset,order_col, value_col):
    w_end = Window.orderBy(order_col).rowsBetween(0,Window.unboundedFollowing)
    dataset = dataset.withColumn(value_col,F.when(F.col(value_col).isNull(),F.first(value_col,True).over(w_end)).otherwise(F.col(value_col)))
    dataset = dataset.select([order_col]+na_cols)
    return dataset
    
for value_col in na_cols:
    dataset = bwfill(dataset, 'Timestamp', value_col)

#### fill with mean values

In [13]:
def fill_mean(dataset,order_col, value_col):
    w_start = Window.orderBy(order_col).rowsBetween(Window.unboundedPreceding,-1)
    # start_val -> last not null value
    dataset = dataset.withColumn('start_val',F.last(value_col,True).over(w_start))
    
    w_end = Window.orderBy(order_col).rowsBetween(0,Window.unboundedFollowing)
    # end_val -> next not null value
    dataset = dataset.withColumn('end_val',F.first(value_col,True).over(w_end))
    
    #fill the null values with the function
    mean_func = (F.col('start_val')+F.col('end_val'))/2
    dataset = dataset.withColumn(value_col,F.when(F.col(value_col).isNull(),mean_func).otherwise(F.col(value_col)))
    
    dataset = dataset.select([order_col]+na_cols)
    return dataset
     
for value_col in na_cols:
    dataset = fill_mean(dataset, 'Timestamp', value_col)

### select time period

In [4]:
from pyspark.sql.functions import col
dataset = dataset.withColumn("Timestamp", col('Timestamp').cast("timestamp"))

In [5]:
# select the time interval
from pyspark.sql.functions import unix_timestamp,from_unixtime
start = F.unix_timestamp(F.lit('2011-12-31 08:00:00')).cast('timestamp')

dataset = dataset.filter(F.col("Timestamp") >= start)

In [8]:
dataset.show(5)

+-------------------+----+----+----+-----+------------------+-----------------+--------------+
|          Timestamp|Open|High| Low|Close|      Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+-------------------+----+----+----+-----+------------------+-----------------+--------------+
|2011-12-31 08:00:00|4.39|4.39|4.39| 4.39|1.2513033658995818|5.493221776299164|          4.39|
|2011-12-31 08:01:00|4.39|4.39|4.39| 4.39|1.3507686778870294|5.929874495924059|          4.39|
|2011-12-31 08:02:00|4.39|4.39|4.39| 4.39| 1.450233989874477|6.366527215548954|          4.39|
|2011-12-31 08:03:00|4.39|4.39|4.39| 4.39|1.5496993018619247| 6.80317993517385|          4.39|
|2011-12-31 08:04:00|4.39|4.39|4.39| 4.39|1.6491646138493725|7.239832654798745|          4.39|
+-------------------+----+----+----+-----+------------------+-----------------+--------------+
only showing top 5 rows



### resample 30min

In [6]:
#add a new column for re-sample purpose
from pyspark.sql import Window
seconds = 60*30
seconds_window = F.from_unixtime(F.unix_timestamp('Timestamp') - F.unix_timestamp('Timestamp') % seconds)
dataset = dataset.withColumn('30_min_window', seconds_window)

In [7]:
# for renaming purpose
cols = dataset.columns
cols = cols[1:-1]
cols

['Open',
 'High',
 'Low',
 'Close',
 'Volume_(BTC)',
 'Volume_(Currency)',
 'Weighted_Price']

In [8]:
#agg and mean all values
dataset = dataset.groupBy('30_min_window').mean()

In [9]:
dataset.printSchema()

root
 |-- 30_min_window: string (nullable = true)
 |-- avg(Open): double (nullable = true)
 |-- avg(High): double (nullable = true)
 |-- avg(Low): double (nullable = true)
 |-- avg(Close): double (nullable = true)
 |-- avg(Volume_(BTC)): double (nullable = true)
 |-- avg(Volume_(Currency)): double (nullable = true)
 |-- avg(Weighted_Price): double (nullable = true)



In [10]:
dataset = dataset.withColumn('Timestamp',F.to_timestamp('30_min_window'))
dataset = dataset.orderBy('Timestamp')

In [39]:
dataset.show(5)

+-------------------+---------+---------+--------+----------+------------------+----------------------+-------------------+-------------------+
|      30_min_window|avg(Open)|avg(High)|avg(Low)|avg(Close)| avg(Volume_(BTC))|avg(Volume_(Currency))|avg(Weighted_Price)|          Timestamp|
+-------------------+---------+---------+--------+----------+------------------+----------------------+-------------------+-------------------+
|2011-12-31 08:00:00|     4.39|     4.39|    4.39|      4.39| 2.693550389717573|    11.824686210860147|               4.39|2011-12-31 08:00:00|
|2011-12-31 08:30:00|     4.39|     4.39|    4.39|      4.39| 5.677509749341007|    24.924267799607012|               4.39|2011-12-31 08:30:00|
|2011-12-31 09:00:00|     4.39|     4.39|    4.39|      4.39| 8.661469108964436|     38.02384938835389|               4.39|2011-12-31 09:00:00|
|2011-12-31 09:30:00|     4.39|     4.39|    4.39|      4.39|11.645428468587864|     51.12343097710073|               4.39|2011-12-31 09

In [11]:
from functools import reduce

old_cols = dataset.columns
old_cols = old_cols[1:-2]

dataset = reduce(lambda dataset, idx: dataset.withColumnRenamed(old_cols[idx], cols[idx]),\
                 range(len(old_cols)), dataset)



In [12]:
dataset = dataset.drop('30_min_window')
dataset.show(5)

+----+----+----+-----+------------------+------------------+-------------------+-------------------+
|Open|High| Low|Close|      Volume_(BTC)| Volume_(Currency)|avg(Weighted_Price)|          Timestamp|
+----+----+----+-----+------------------+------------------+-------------------+-------------------+
|4.39|4.39|4.39| 4.39| 2.693550389717573|11.824686210860147|               4.39|2011-12-31 08:00:00|
|4.39|4.39|4.39| 4.39| 5.677509749341007|24.924267799607012|               4.39|2011-12-31 08:30:00|
|4.39|4.39|4.39| 4.39| 8.661469108964436| 38.02384938835389|               4.39|2011-12-31 09:00:00|
|4.39|4.39|4.39| 4.39|11.645428468587864| 51.12343097710073|               4.39|2011-12-31 09:30:00|
|4.39|4.39|4.39| 4.39|14.629387828211296|  64.2230125658476|               4.39|2011-12-31 10:00:00|
+----+----+----+-----+------------------+------------------+-------------------+-------------------+
only showing top 5 rows

